---
**Movie Recommendation System**
---

---
**Initial Configurations**
---

In [127]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [128]:
movies_data = pd.read_csv('./Movie-Recommendation-System-Dataset/movies.csv')

In [129]:
# Dataset
print(f"Dataset shape: {movies_data.shape}")
movies_data.head(3)

Dataset shape: (4803, 24)


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [130]:
#data info
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

---
**Data Preparation**
---

In [131]:
#Selecting the relevant features for the recommendation system
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(f"Selected features: {selected_features}")

Selected features: ['genres', 'keywords', 'tagline', 'cast', 'director']


In [132]:
# Replacing the NULL values with NULL String

for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')
    

In [133]:
# Combining all the 5 selected features

combined_features =  movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']

In [134]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [135]:
# Converting the text data to the numerical data for finding cosine similarity

vectorizer = TfidfVectorizer()

In [136]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [137]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

---
**Cosine Similarity**
---

In [138]:
# Getting Similarity Scores / Similarity Confidence

similarity = cosine_similarity(feature_vectors)

In [139]:
print("Cosine Similarity Matrix Shape : ", similarity.shape, "\n")
print(similarity)

Cosine Similarity Matrix Shape :  (4803, 4803) 

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


---
**Getting the Movie Name**
---

In [140]:
movie_name = input("Enter your favourite movie name : ")
print("Movie Name : ", movie_name)

Movie Name :  iron man


In [141]:
# Creating a list with all the movie names given in the dataset
all_titles_list = movies_data['title'].tolist()

In [142]:
# Finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name,all_titles_list)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [143]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [144]:
# Finding the index of the movie with title

index_of_movie = movies_data[movies_data.title == close_match]['index'].values[0]

print(index_of_movie)


68


In [145]:
# Getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_movie]))
# print(f"Movie Name : {close_match} |  Similarity Score : {similarity_score}")

In [146]:
print(len(similarity_score))

4803


In [147]:
# Sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key= lambda x : x[1], reverse=True)
# print(sorted_similar_movies)

In [148]:
# Print top 10 matches

print("Movies suggested for you : \n")
i = 0
for movie in sorted_similar_movies : 
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if(i < 20):
        print(i+1, '.', title_from_index)
        i += 1
    else :
        break

    


Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X-Men: Apocalypse
12 . X2
13 . The Incredible Hulk
14 . The Helix... Loaded
15 . X-Men: First Class
16 . X-Men: Days of Future Past
17 . Captain America: The First Avenger
18 . Kick-Ass 2
19 . Guardians of the Galaxy
20 . Deadpool
